# Размер случайного леса

Данное задание основано на материалах лекций по логическим методам и направлено на знакомство со случайными лесами (Random Forests).
## Вы научитесь:

• работать со случайным лесом — одним из наиболее распространенных семейств алгоритмов

• решать с его помощью задачи регрессии

• подбирать параметры случайного леса

## Введение
Случайный лес — это модель классификации, объединяющая некоторое количество решающих деревьев в одну композицию, за счет чего улучшается их качество работы и обобщающая способность. Деревья строятся независимо друг от друга. Чтобы они отличались друг от друга, обучение проводится не на всей обучающей выборке, а на ее случайном подмножестве. Также, для дальнейшего уменьшения схожести деревьев, оптимальный признак для разбиения выбирается не из всех возможных признаков, а лишь из их случайного подмножества. Прогнозы, выданные деревьями, объединяются в один ответ путем усреднения.

Особенность случайного леса заключается в том, что он не переобучается по мере увеличения количества деревьев в композиции. Это достигается за счет того, что деревья не зависят друг от друга, и поэтому добавление нового дерева в композицию не усложняет модель, а лишь понижает уровень шума в прогнозах.

## Реализация в Scikit-Learn
Загрузим датасет abalone.csv. Это датасет, в котором требуется предсказать возраст ракушки (число колец) по физическим измерениям.

In [16]:
import pandas as pd
data = pd.read_csv('abalone.csv') 

Преобразуем признак Sex в числовой: значение F должно перейти в -1, I — в 0, M — в 1. 

In [17]:
data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))

Разделим содержимое файлов на признаки и целевую переменную. В последнем столбце записана целевая переменная, в остальных — признаки.

In [18]:
train_X=data.iloc[:,:-1].values
train_Y=data.iloc[:,-1:].values.ravel()

В качестве меры качества воспользуемся коэффициентом детерминации (sklearn.metrics.r2_score):

In [19]:
from sklearn.metrics import make_scorer
from  sklearn.metrics import r2_score
ms=make_scorer(r2_score)
validationTest={}

Составим конструктор кросс-валидации по 5 блокам

In [20]:
from sklearn.model_selection import KFold
kf = KFold(random_state=1,shuffle=True)

Обучим случайный лес с различным числом деревьев: от 1 до 50. Для каждого варианта оценим качество работы полученного леса на кросс-валидации по 5 блокам и определим, при каком минимальном количестве деревьев случайный лес показывает качество
на кросс-валидации выше 0.52.

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
for N in range(50):
    n_estimators=N+1
    regressor = RandomForestRegressor(n_estimators,random_state=1)
    scores = cross_val_score(regressor, train_X, train_Y, scoring=ms ,cv=kf)
    val=scores.mean()
    if val>0.52:
        validationTest[n_estimators]=val

In [22]:
print(f"Минимальное количество деревьев: {min(validationTest.keys())}")

Минимальное количество деревьев: 21
